In [38]:

# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


In [3]:
import numpy as np
import pandas as pd

In [1]:
%%bigquery df
select * from bqml.sensor_data

Downloading: 100%|██████████| 75000/75000 [00:02<00:00, 30801.49rows/s]


In [2]:
df.columns

Index(['dttm', 'id', 'val'], dtype='object')

In [10]:
df['id'] = df['id'].str.replace('-','_')

In [11]:
df_pvt = pd.pivot_table(df, values='val', index=['dttm'],
                    columns=['id'], aggfunc=np.average)

In [12]:
df_pvt.head()

id,sensor_1,sensor_10,sensor_100,sensor_1000,sensor_1001,sensor_1002,sensor_1003,sensor_1004,sensor_1005,sensor_1006,...,sensor_990,sensor_991,sensor_992,sensor_993,sensor_994,sensor_995,sensor_996,sensor_997,sensor_998,sensor_999
dttm,,,,,,,,,,,,,,,,,,,,,
2021-07-19 00:00:00+00:00,9,8,4,3,9,6,5,9,7,2,...,3,8,7,7,2,5,0,7,9,3
2021-07-19 01:00:00+00:00,5,1,2,9,7,3,3,2,6,6,...,6,8,6,8,1,7,1,4,9,2
2021-07-19 02:00:00+00:00,6,9,5,6,8,4,7,2,10,9,...,4,4,10,4,4,9,2,3,0,1
2021-07-19 03:00:00+00:00,2,8,1,9,1,0,1,4,9,1,...,9,0,3,6,1,5,1,6,9,9
2021-07-19 04:00:00+00:00,3,6,1,9,9,3,6,5,3,2,...,8,6,4,4,2,0,7,8,10,6


In [13]:
df_pvt.to_gbq('bqml.sensor_data_pvt',if_exists='replace')

1it [00:07,  7.23s/it]


In [14]:
%%bigquery 
CREATE MODEL bqml.sensor_predict_value
OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
        BOOSTER_TYPE = 'GBTREE',
        NUM_PARALLEL_TREE = 1,
        MAX_ITERATIONS = 50,
        TREE_METHOD = 'HIST',
        EARLY_STOP = FALSE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['sensor_3000'])
AS SELECT * FROM bqml.sensor_data_pvt;

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1551.91query/s]                        


""


In [19]:
%%bigquery explain_df
select * from ML.FEATURE_IMPORTANCE(MODEL `bqml.sensor_predict_value`)
order by 2 desc,3 desc,4 desc

Downloading: 100%|██████████| 2999/2999 [00:02<00:00, 1495.27rows/s]


In [22]:
explain_df.head()

,feature,importance_weight,importance_gain,importance_cover
0,sensor_1104,3,0.902998,3.898307
1,sensor_1,3,0.436065,3.840000
2,sensor_1608,2,6.318995,4.405050
3,sensor_1693,2,3.395460,4.788730
4,sensor_241,2,1.737424,3.779885


In [21]:
explain_df.importance_weight.value_counts()

0    2851
1     130
2      16
3       2
Name: importance_weight, dtype: int64

In [31]:
%%bigquery
CREATE MODEL bqml.sensor_fcst
 OPTIONS(MODEL_TYPE='ARIMA_PLUS',
         time_series_timestamp_col='dttm',
         time_series_data_col='val',
        data_frequency = 'HOURLY',
        horizon = 4,
        time_series_id_col='id') AS
SELECT
  dttm,
  id,
  val
FROM
  bqml.sensor_data
  
WHERE id IN (
    select replace(feature,'_','-')  from ML.FEATURE_IMPORTANCE(MODEL `bqml.sensor_predict_value`)
    WHERE importance_weight > 0 
)

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1753.47query/s]                        


""


In [32]:
%%bigquery
SELECT
  *
FROM
  ML.FORECAST(MODEL bqml.sensor_fcst,
              STRUCT(4 AS horizon, 0.8 AS confidence_level))

Downloading: 100%|██████████| 592/592 [00:02<00:00, 273.07rows/s]


,id,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,sensor-1,2021-07-20 01:00:00+00:00,4.404177,2.170756,0.8,1.620645,7.187708,1.620645,7.187708
1,sensor-1,2021-07-20 02:00:00+00:00,5.402823,2.231877,0.8,2.540917,8.264729,2.540917,8.264729
2,sensor-1,2021-07-20 03:00:00+00:00,2.764239,2.675596,0.8,-0.666642,6.195119,-0.666642,6.195119
3,sensor-1,2021-07-20 04:00:00+00:00,5.511900,2.676489,0.8,2.079874,8.943925,2.079874,8.943925
4,sensor-1001,2021-07-20 01:00:00+00:00,2.993941,2.343103,0.8,-0.010589,5.998470,-0.010589,5.998470
...,...,...,...,...,...,...,...,...,...
587,sensor-944,2021-07-20 04:00:00+00:00,5.000000,3.006659,0.8,1.144601,8.855399,1.144601,8.855399
588,sensor-974,2021-07-20 01:00:00+00:00,6.009860,2.427131,0.8,2.897582,9.122137,2.897582,9.122137
589,sensor-974,2021-07-20 02:00:00+00:00,3.756598,2.476193,0.8,0.581409,6.931787,0.581409,6.931787
590,sensor-974,2021-07-20 03:00:00+00:00,5.221563,2.484577,0.8,2.035624,8.407503,2.035624,8.407503


In [35]:
%%bigquery
SELECT
  *
FROM
  ML.DETECT_ANOMALIES(MODEL bqml.sensor_fcst,
        STRUCT(0.9 AS anomaly_prob_threshold),
                      (
                       SELECT
                          forecast_timestamp as dttm,
                          id,
                          forecast_value as val
                        FROM
                          ML.FORECAST(MODEL bqml.sensor_fcst,
                                      STRUCT(4 AS horizon, 0.8 AS confidence_level))))
    

Downloading: 100%|██████████| 592/592 [00:02<00:00, 288.43rows/s]


,id,dttm,val,is_anomaly,lower_bound,upper_bound,anomaly_probability
0,sensor-1614,2021-07-20 01:00:00+00:00,4.920000,False,-0.478083,10.318083,0.0
1,sensor-1614,2021-07-20 02:00:00+00:00,4.920000,False,-0.478083,10.318083,0.0
2,sensor-1614,2021-07-20 03:00:00+00:00,4.920000,False,-0.478083,10.318083,0.0
3,sensor-1614,2021-07-20 04:00:00+00:00,4.920000,False,-0.478083,10.318083,0.0
4,sensor-1964,2021-07-20 01:00:00+00:00,5.280000,False,-0.069856,10.629856,0.0
...,...,...,...,...,...,...,...
587,sensor-1965,2021-07-20 04:00:00+00:00,4.880000,False,-0.023294,9.783294,0.0
588,sensor-295,2021-07-20 01:00:00+00:00,4.949531,False,1.078690,8.820372,0.0
589,sensor-295,2021-07-20 02:00:00+00:00,4.974637,False,1.101290,8.847984,0.0
590,sensor-295,2021-07-20 03:00:00+00:00,5.049648,False,0.982001,9.117295,0.0
